In [2]:
import os
import pandas as pd
import numpy as np
import requests 
import pickle
from unidecode import unidecode
import requests, urllib
from bs4 import BeautifulSoup
from selenium import webdriver
from helpers import *

In [11]:
companies = pd.read_excel("../data/MSCI/raw/basic_listed_mt500.xls")

WARNING *** file size (647967) not 512 + multiple of sector size (512)


# Step 1: Find Identifier for each Company

In [3]:
msci_identifiers = get_identifiers(companies, start_from_company=False)
msci_identifiers_df = pd.DataFrame(msci_identifiers)
msci_identifiers_df.to_csv("../data/MSCI/identifiers/treatment_company_identifiers.csv", index=False)

ROYAL DUTCH SHELL PLC 1 of 0 found
ASTRAZENECA PLC 90 of 100 found
PUBLICIS GROUPE S A 169 of 200 found
FINCANTIERI S.P.A. 230 of 300 found
CHOCOLADEFABRIKEN LINDT & SPRÜNGLI AG 280 of 400 found
GATES INDUSTRIAL CORPORATION PLC 314 of 500 found
ΕΛΙΝΟΙΛ ΕΛΛΗΝΙΚΗ ΕΤΑΙΡΕΙΑ ΠΕΤΡΕΛΑΙΩΝ Α.Ε. 345 of 600 found
YANDEX N.V. 382 of 700 found
ÅF PÖYRY AB 412 of 800 found
FERREXPO PLC 440 of 900 found
NEXI SPA 465 of 1000 found
TED BAKER PLC 493 of 1100 found
EXPRIVIA S.P.A. IN FORMA ALTERNATIVA: AIS S.P.A., AISOFTW@RE S.P. A., ARTIFICIAL INTELLIGENCE SOFTWARE S.P.A. 507 of 1200 found
WYNNSTAY GROUP P.L.C. 530 of 1300 found
DUKAT D.D. 547 of 1400 found
NMCN PLC 565 of 1500 found
CENTRUM NOWOCZESNYCH TECHNOLOGII S.A. 584 of 1600 found
ΠΛΑΣΤΙΚΑ ΚΡΗΤΗΣ Α.Β.Ε.Ε. 598 of 1700 found
ORELL FÜSSLI HOLDING AG 610 of 1800 found
AMADEUS FIRE AG 632 of 1900 found
JACQUES BOGART SA 653 of 2000 found
FJORD LINE AS 668 of 2100 found
WIRTUALNA POLSKA HOLDING S.A. 686 of 2200 found
ERLUS AKTIENGESELLSCHAFT 699 of 23

# Step 2: Match Identifiers with Amadeus Data

In [12]:
msci_identifiers = pd.read_csv("../data/MSCI/identifiers/treatment_company_identifiers.csv").drop_duplicates()

msci_identifiers["title_cleaned"] = msci_identifiers["title"].apply(lambda x: standardize_string(x))
companies["title_cleaned"] = companies["Company name"].apply(lambda x: standardize_string(x))

print len(set(msci_identifiers["title_cleaned"]).intersection(set(companies["title_cleaned"])))

merged_dataframe = companies.merge(msci_identifiers, how='left', on="title_cleaned")

276


# Step 3: Getting actual Data from MSCI 

In [13]:
merged_dataframe = merged_dataframe[["encodedTitle", "url"]].dropna()
data_extract = extract_data(merged_dataframe)

IID000000002137074 royal-dutch-shell-plc
IID000000002140371 bp-plc
IID000000002619434 glencore-plc
IID000000002163566 daimler-ag
IID000000002145155 total-sa
IID000000002784170 exor-nv
IID000000002148933 bayerische-motoren-werke-aktiengesellschaft
IID000000002769077 uniper-se
IID000000002183691 siemens-aktiengesellschaft
IID000000002163952 deutsche-telekom-ag
IID000000002151160 enel-spa
IID000000002164704 eni-spa
IID000000002153923 tesco-plc
IID000000002125950 airbus-se
IID000000002158653 basf-se
IID000000002163947 deutsche-post-ag
IID000000002172935 koninklijke-ahold-delhaize-nv
IID000000002157753 arcelormittal-sa
IID000000002181589 roche-holding-ag
IID000000002184880 equinor-asa
IID000000002187964 unilever-nv
IID000000002181334 repsol-sa
IID000000002137159 unilever-plc
IID000000002186444 telefonica-sa
IID000000002144457 novartis-ag
IID000000002148934 continental-aktiengesellschaft
IID000000002180858 rwe-aktiengesellschaft
IID000000002123944 bayer-aktiengesellschaft
IID000000002189051 

IID000000002128288 smiths-group-plc
IID000000002716127 informa-plc
IID000000002672787 fuchs-petrolub-se
IID000000002184374 sonova-holding-ag
IID000000002130138 eon-se
IID000000002140364 biomerieux-sa
IID000000002168620 h-lundbeck-a/s
IID000000002142057 coloplast-a/s
IID000000002129162 whitbread-plc
IID000000002676654 london-stock-exchange-group-plc
IID000000002186408 tele2-ab
IID000000002127391 meggitt-plc
IID000000002182455 the-sage-group-plc
IID000000002676862 allegion-public-limited-company
IID000000002137341 united-utilities-group-plc
IID000000002189667 wirecard-ag
IID000000002151050 ems-chemie-holding-ag
IID000000002137326 severn-trent-plc
IID000000002145368 red-electrica-corporacion-sa
IID000000002152916 novozymes-a/s
IID000000002128271 demant-a/s
IID000000002596635 ocado-group-plc
IID000000002643578 nmc-health-plc
IID000000002165151 elisa-oyj
IID000000002153893 schibsted-asa
IID000000002185460 swedish-match-ab
IID000000002631142 polymetal-international-plc
IID000000002182746 sar

# Step 4: Create Backup

In [14]:
data_extract_df = pd.DataFrame(data_extract).T
data_extract_df.columns = [c.strip() for c in data_extract_df.columns]

data_extract_df = clean_df(data_extract_df)

data_extract_df.astype("str").applymap(lambda x: x.encode('utf-8', 'ignore')).to_csv("../data/MSCI/scrapped/treatment_data_extract.csv")

# Step 5: Extract Useful Information

#####  ['Access to Communication','Access to Finance','Access to Health Care','Biodiversity & Land Use','Carbon Emissions','Chemical Safety','Controversial Sourcing','Corporate Governance','Corruption & Instability','Electronic Waste','Financial Product Safety','Health & Safety','Human Capital Development','Insuring Health & Demographic Risk','Labor Management','Opportunities in Clean Tech','Opportunities in Green Building','Opportunities in Nutrition & Health','Opportunities in Renewable Energy','Packaging Material & Waste','Privacy & Data Security','Product Carbon Footprint','Product Safety & Quality','Raw Material Sourcing','Responsible Investment','Supply Chain Labor Standards','Toxic Emissions & Waste','Water Stress']

In [15]:
data_extract_df = pd.read_csv("../data/MSCI/scrapped/treatment_data_extract.csv")

id_date_rating_pairs = data_extract_df.apply(lambda x: zip([x["Unnamed: 0"]]*len(x["date"]),[x["country"]]*len(x["date"]),[x["industry"]]*len(x["date"]), x["date"].split(","), x["rating_history"].split(",")),axis=1)
esg_rating_panel = pd.DataFrame([item for sublist in id_date_rating_pairs.values.tolist() for item in sublist],columns = ["company_id","country","industry","date","esg_rating"])
esg_rating_panel.to_csv("../data/MSCI/panel/treatment_firms_esg_rating_panel.csv", index = False)